In [1]:
%load_ext autoreload
%autoreload 2
from TNR import *

In [95]:
# quantum 1d ising:
J=1
htemp=KP(sz,sz)+J*(KP(sx,s0)+KP(s0,sx))/2
chi={'w':6,'y':8,'u':6,'v':6,'k':8,'q':36}
coarsensteps=10

# quantum 1d zeeman
# htemp=(KP(sz,s0)+KP(s0,sz))/2+(KP(sx,s0)+KP(s0,sx))/2
# chi={'w':4,'y':2,'u':2,'v':2}
# coarsensteps=10

# quantum XY:
# J=1
# htemp=KP(sx,sx)+KP(sz,sz)
# chi={'w':12,'y':8,'u':8,'v':8}
# coarsensteps=8
# htemp=(KP(s0,sx,sx,s0)+KP(s0,sz,sz,s0))+J*(KP(sx,sx,s0,s0)+KP(sz,sz,s0,s0)+KP(s0,s0,sx,sx)+KP(s0,s0,sz,sz))/2

# quantum 1d Potts
# J=1
# htemp=-KP(R,R.conj())-J*(KP(M,np.eye(3))+KP(np.eye(3),M))/2
# htemp+=htemp.T.conj()
# chi={'w':12,'y':9,'u':9,'v':9,'k':15,'q':66}
# coarsensteps=9
# RGsteps=5

# quantum 1d z3z3
# htemp=-KP(R,R.conj())-KP(M,M)
# htemp+=htemp.T.conj()
# coarsensteps=8

#quantum 1d pre-steps
d=int(np.sqrt(htemp.shape[0]))
Ainit=expm(-0.002*htemp).reshape(d,d,d,d)
Asplit_tensors,A00=Asplit(Ainit,chi)
coarsen_list=[{'A':A00}]
for i in range(coarsensteps):
    Aold=coarsen_list[i]['A']
    coarsen_list[i],Acoarse=coarsen(Aold,chi)
    coarsen_list.append({'A':Acoarse})
A0=coarsen_list[-1]['A']

# classical ising:
# chi={'w':6,'y':6,'u':6,'v':6,'k':12,'q':24}
# na=np.newaxis
# s=np.array([1,-1])
# Tc=2/np.log(1+np.sqrt(2))
# T=1*Tc
# A0=np.exp(-1/T*(s[:,na,na,na]*s[na,:,na,na]+s[na,:,na,na]*s[na,na,:,na]\
#                 +s[na,na,:,na]*s[na,na,na,:]+s[na,na,na,:]*s[:,na,na,na]))
# A0/=np.linalg.norm(A0)

tensors_list=[{'A':A0}]
chiw,chiy,chiu,chiv=chi['w'],chi['y'],chi['u'],chi['v']

vertical truncation err: 0.0
horizontal truncation err: 1.3307111039703433e-12
Aout shape: (3, 4, 3, 4)
Coarsen truncation error: 6.63999e-11 Acoarse shape: (3, 4, 3, 4)
Coarsen truncation error: 5.52341e-13 Acoarse shape: (4, 4, 4, 4)
Coarsen truncation error: 2.07825e-11 Acoarse shape: (4, 4, 4, 4)
Coarsen truncation error: 2.11725e-11 Acoarse shape: (5, 4, 5, 4)
Coarsen truncation error: 4.44935e-12 Acoarse shape: (6, 4, 6, 4)
Coarsen truncation error: 8.79816e-11 Acoarse shape: (8, 4, 8, 4)
Coarsen truncation error: 2.3348e-08 Acoarse shape: (8, 4, 8, 4)
Coarsen truncation error: 3.26661e-06 Acoarse shape: (8, 4, 8, 4)
Coarsen truncation error: 0.000170188 Acoarse shape: (8, 4, 8, 4)
Coarsen truncation error: 0.00372825 Acoarse shape: (8, 4, 8, 4)


In [96]:
RGsteps=5
i0=len(tensors_list)-1
for i in range(RGsteps):
    print('RG step: ',i0+i)
    Aold=tensors_list[i0+i]['A']
    tensors_list[i0+i],Anew=doTNR(Aold,chi)
    tensors_list.append({'A':Anew})
    print('A.shape=',Anew.shape)
    print('chiv=',tensors_list[i0+i]['vl'].shape[2])
    print()

RG step:  0
iter=0, U.Uenv=0.393018
iter=100, U.Uenv=0.393249
iter=200, U.Uenv=0.393249
error in U, vl, vr optimization: 0.0012854
error in yl, yr, D optimization: 0.010169
error in w optimization: 0.0079128
iter=0, u.uenv=0.863634, v.venv=0.78601
iter=1000, u.uenv=0.996602, v.venv=0.996602
iter=2000, u.uenv=0.996602, v.venv=0.996602
Adiff 0.0739625
A.shape= (8, 6, 8, 6)
chiv= 6

RG step:  1
iter=0, U.Uenv=0.414228
iter=100, U.Uenv=0.414316
iter=200, U.Uenv=0.414317
iter=300, U.Uenv=0.414317
iter=400, U.Uenv=0.414317
error in U, vl, vr optimization: 0.00142528
error in yl, yr, D optimization: 0.0105415
error in w optimization: 0.00745651
iter=0, u.uenv=0.933104, v.venv=0.815065
iter=1000, u.uenv=0.999926, v.venv=0.999926
iter=2000, u.uenv=0.999926, v.venv=0.999926
Adiff 0.0121384
A.shape= (8, 6, 8, 6)
chiv= 6

RG step:  2
iter=0, U.Uenv=0.411154
iter=100, U.Uenv=0.411262
iter=200, U.Uenv=0.411262
iter=300, U.Uenv=0.411262
iter=400, U.Uenv=0.411262
error in U, vl, vr optimization: 0.001

In [23]:
i=3
A=tensors_list[i+1]['A']
from scipy.sparse.linalg import eigsh

In [24]:
T1=np.einsum(A,[2,1,2,0])
# ew,_=np.linalg.eigh(T)
ew = eigsh(T1, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[2]-el[0]))[:100]

c:\programdata\miniconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1254: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  RuntimeWarning)


array([-0.        ,  0.1185881 ,  1.        ,  1.03526507,  1.11012191,
        1.42195287])

In [25]:
T2=np.einsum(A,[4,2,5,0],A,[5,3,4,1]).reshape(chiw**2,chiw**2)
ew = eigsh(T2, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[2]-el[0]))[:100]

c:\programdata\miniconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1254: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  RuntimeWarning)


array([-0.        ,  0.12498022,  1.        ,  1.12335006,  1.12421145,
        1.98774168,  2.00162317,  2.02697953,  2.04074836,  2.14480648,
        2.14828669,  2.17457332,  2.70948425,  2.7492312 ,  2.7497438 ,
        2.88486088,  3.0478844 ,  3.21521456,  3.30954822,  3.31570005,
        3.35564174,  3.44284917,  3.79468675,  4.03144511,  4.08477725,
        4.09811851,  4.1025944 ,  4.55638605,  4.56196265,  4.42458202,
        4.22736376,  4.11736855,  3.88423798,  3.5024575 ,  3.31580867,
        3.27952689])

In [26]:
T3=np.einsum(A,[10,3,8,0],A,[8,4,9,1],A,[9,5,10,2]).reshape(chiw**3,chiw**3)
ew = eigsh(T3, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[2]-el[0]))[:100]

array([-0.        ,  0.12557751,  1.        ,  1.12417724,  1.12417724,
        1.99928523,  1.99928523,  2.00783879,  2.00783879,  2.13305496,
        2.13305496,  2.13391197,  2.99484746,  3.02226469,  3.04008761,
        3.04008761,  3.04803807,  3.11777385,  3.12159258,  3.157412  ,
        3.16859833,  3.16859833,  3.21063772,  3.90421335,  3.91345114,
        3.91345114,  3.94182719,  3.94182719,  4.04721177,  4.04721177,
        4.05701377,  4.1283819 ,  4.1283819 ,  4.1734247 ,  4.1734247 ,
        4.18636669,  4.18636669,  4.30269029,  4.36442643,  4.45024191,
        4.45024191,  4.46927825,  4.46927825,  4.62702991,  4.843758  ,
        4.843758  ,  4.97509996,  4.98832185,  4.98832185,  5.13513025,
        5.13513025,  5.22219337,  5.24624473,  5.24624473,  5.34194669,
        5.34194669,  5.34585416,  5.36877996,  5.36877996,  5.40721602,
        5.40956881,  5.53567869,  5.53567869,  5.65783339,  5.65783339,
        5.67619994,  5.74340877,  5.74340877,  5.78810004,  5.78

In [27]:
T4=np.einsum(A,[11,4,8,0],A,[8,5,9,1],A,[9,6,10,2],A,[10,7,11,3]).reshape(chiw**4,chiw**4)
ew = eigsh(T4, k=100, which='LM', return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[2]-el[0]))[:100]

array([-0.        ,  0.12591164,  1.        ,  1.12397447,  1.12397447,
        1.99983768,  1.99997458,  2.00306896,  2.00306896,  2.12643182,
        2.12905483,  2.12944401,  3.00453135,  3.00453135,  3.01254739,
        3.01569266,  3.01691209,  3.12568275,  3.12568275,  3.1394174 ,
        3.1394174 ,  3.14532227,  3.14532227,  3.99490718,  4.00612003,
        4.01344961,  4.02162664,  4.04391256,  4.04391256,  4.04505071,
        4.05574914,  4.05574914,  4.11939986,  4.12362097,  4.1444107 ,
        4.14449174,  4.16025206,  4.16318961,  4.17766789,  4.18094592,
        4.26441083,  4.96637003,  4.96637003,  4.98545589,  5.01687769,
        5.01959283,  5.03656792,  5.03656792,  5.05780751,  5.05780751,
        5.06368734,  5.10555148,  5.10555148,  5.12579794,  5.13420632,
        5.14528313,  5.14528313,  5.16011322,  5.16011322,  5.18305417,
        5.18305417,  5.1918607 ,  5.1918607 ,  5.2562973 ,  5.2562973 ,
        5.35169484,  5.45592533,  5.52237149,  5.52237149,  5.53

In [28]:
T4reduced=np.einsum(A,[11,0,8,0],A,[8,5,9,1],A,[9,6,10,2],A,[10,3,11,3]).reshape(chiw**2,chiw**2)
ew = eigsh(T4reduced, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[9]-el[0]))[:100]

c:\programdata\miniconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1254: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  RuntimeWarning)


array([-0.        ,  0.09351399,  0.37195871,  0.44375947,  0.56315653,
        0.72260528,  0.84655332,  0.89011332,  0.94521416,  1.        ,
        1.0932012 ,  1.11792341,  1.22602407,  1.28395641,  1.41493762,
        1.43444532,  1.451283  ,  1.49775921,  1.55779045,  1.57067217,
        1.59382801,  1.72349635,  1.79866985,  1.80214377,  1.8446175 ,
        1.87493968,  2.05647616,  2.15012709,  2.19710137,  2.2712998 ,
        2.3083409 ,  2.35124155,  2.4235692 ,  2.4850527 ,  1.97600602,
        1.88467137])

In [7]:
#eval mag for quantum 1d models
op=sz
MOq=MOq_(op,Asplit_tensors,coarsen_list)
eval_op(MOq,tensors_list)  

RGsteps: 0 , <O> = (-0.9486832980504737+0j)
RGsteps: 1 , <O> = (-0.9486832980504737+0j)
RGsteps: 2 , <O> = (-0.9486832980504737+0j)
RGsteps: 3 , <O> = (-0.9486832980504737+0j)
RGsteps: 4 , <O> = (-0.9486832980504737+0j)


(-0.9486832980504737+0j)

In [29]:
#eval mag for classical ising:
op=sz
A0=tensors_list[0]['A']
MO=MO_(A0,op)
eval_op(MO,tensors_list)

RGsteps: 0 , <O> = 0j
RGsteps: 1 , <O> = 0j
RGsteps: 2 , <O> = 0j
RGsteps: 3 , <O> = 0j
RGsteps: 4 , <O> = (-6.392594011758512e-10+0j)


(-6.392594011758512e-10+5.068979602773256e-20j)

In [47]:
#exact value
mag_exact=(1-np.sinh(2/T)**(-4))**(1/8)
print(mag_exact)

0.629648162973007


scaling dimensions

In [97]:
i=3
tensors=tensors_list[i]
Aold=tensors['A'];
chiwt=Aold.shape[1]
chiyt=Aold.shape[0]
scale_tensors=scale_tensors_(tensors)

def scaleop(v):
    M=v.reshape(chiwt,chiwt,chiyt,chiyt,chiyt,chiyt,chiwt,chiwt)
    Mscaled=rg_(M,scale_tensors)
    return Mscaled.flatten()

n=chiw**4*chiy**4

In [101]:
# compress legs

chik=8
chiq=36
gu,gr,gl,gnw,gne=scale_tensors['gu'],scale_tensors['gr'],scale_tensors['gl'],scale_tensors['gnw'],scale_tensors['gne']
guw,glw,grw,wv,wr,wl = g_compressed(Aold,gu,gr,gl,chik,chiq)

def scaleop(v):
    v4=v.reshape(chiq,chik,chiq,chik)
    out=ncon(v4,[8,9,10,11],glw,[10,19,21],grw,[8,18,20],guw,[9,22,23],guw.conj(),[11,16,17],gnw,[20,22,5,14],gne,[21,23,7,15],gnw.conj(),[18,16,4,12],gne.conj(),[19,17,6,13],wr,[4,5,0],wl,[6,7,2],wv,[14,15,1],wv.conj(),[12,13,3])
    return out.flatten()
n=chik**2*chiq**2

gu compression error: 0.000506281642929517
gr compression error: 1.980180723666667e-16
gl compression error: 0.0


In [55]:
import scipy
from scipy.sparse.linalg import LinearOperator

N_level=40
Atemp = LinearOperator((n,n), matvec = scaleop, dtype='complex')
ew = scipy.sparse.linalg.eigs(Atemp, k=N_level, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew=np.abs(ew)
ew.sort()
ew=ew[::-1]
scaledim=-np.log2(np.abs(ew/ew[0]))
scaledim

array([-0.        ,  0.12655181,  1.00131558,  1.12441053,  1.12494036,
        2.00148834,  2.00177471,  2.00229527,  2.00361926,  2.12322129,
        2.126631  ,  2.13012912,  3.0005998 ,  3.00263808,  3.00327742,
        3.00459674,  3.00753422,  3.10694089,  3.12905695,  3.12990191,
        3.13224755,  3.13453359,  3.13951972,  3.99620319,  3.99759564,
        4.00029876,  4.00270958,  4.0049498 ,  4.0056991 ,  4.010566  ,
        4.01187323,  4.01443199,  4.06879442,  4.1188085 ,  4.12315199,
        4.13281048,  4.13309002,  4.13969317,  4.14728885,  4.14811996])

In [72]:
i=3
tensors=tensors_list[i]
U=tensors['U'];w=tensors['w'];u=tensors['u']

wgauged=w@u
# wgauged=w

def scaleop(v):
    M=v.reshape(chiw,chiw,chiw,chiw)
    mera=ncon(M,[0,1,2,3],U,[0,1,8,9],wgauged,[12,8,4],wgauged,[9,13,5],U.conj(),[2,3,10,11],wgauged.conj(),[12,10,6],wgauged.conj(),[11,13,7])
    return mera.flatten()

n=chiw**4

In [73]:
import scipy
from scipy.sparse.linalg import LinearOperator

N_level=25
Atemp = LinearOperator((n,n), matvec = scaleop, dtype='complex')
ew = scipy.sparse.linalg.eigs(Atemp, k=N_level, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
-np.log2(np.abs(ew/ew[0]))

array([-0.        ,  0.12728364,  0.9684625 ,  1.10353051,  1.20739804,
        1.20739804,  1.59860124,  1.59860124,  1.64755579,  1.8180896 ,
        1.8180896 ,  1.81271592,  1.81271592,  1.81871814,  1.81871814,
        1.92904183,  1.98387286,  2.07673273,  2.27317919,  2.35460655,
        2.46307274,  2.49162207,  2.53869677,  2.52851065,  2.52851065])

In [77]:
data={'chi':chi,'tensors_list':tensors_list}
np.save('ising',data)

In [78]:
data=np.load('ising.npy')

In [82]:
data2=dict(data)

TypeError: iteration over a 0-d array

In [86]:
import pickle
with open('data.pkl','rb') as f:
    data=pickle.load(f)

In [87]:
data.keys()

dict_keys(['chi', 'tensors_list'])